In [2]:
# import modules
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer 
import re, nltk, numpy, matplotlib
import pandas as pd
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer 
from sklearn.decomposition import LatentDirichletAllocation
from gensim import corpora
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
from datetime import datetime
import string
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>


In [3]:
df = pd.read_csv('C:/Users/2146806A/OneDrive - University of Glasgow/University of Glasgow/Amsterdam Visit/GITHUB FILES/Data/preprocessed.csv', encoding='latin-1')
df.head()  

,date,maintext,title,source,media_name,ideology,Congress,text_lemma,title_lemma,maintext_no_fullstop
0,03/01/2013,greg gutfeld cohost america. greg gutfeld andr...,reagan era al jazeera buys current tv,C:/Users/2146806A/OneDrive - University of Gla...,Fox News,right,113th,greg gutfeld cohost america . greg gutfeld and...,reagan era al jazeera buy current tv,greg gutfeld cohost america greg gutfeld andre...
1,11/01/2013,latest threat leadership easily speaker house ...,job safe straus challenge gop,C:/Users/2146806A/OneDrive - University of Gla...,The New York Times,left,113th,late threat leadership easily speaker house jo...,job safe straus challenge gop,latest threat leadership easily speaker house ...
2,18/01/2013,story highlights mills remembers march selma m...,years mlk march voting rights vulnerable,https://edition.cnn.com/2013/01/18/opinion/mil...,CNN,left,113th,story highlight mill remember march selma mont...,year mlk march vote right vulnerable,story highlights mills remembers march selma m...
3,20/01/2013,barack michelle obama spent thousand days disp...,change years friends shifts obamas,C:/Users/2146806A/OneDrive - University of Gla...,The New York Times,left,113th,barack michelle obama spend thousand day displ...,change year friend shift obama,barack michelle obama spent thousand days disp...
4,21/01/2013,sean hannity host hannity moments ago presiden...,interview karl rove,C:/Users/2146806A/OneDrive - University of Gla...,Fox News,right,113th,sean hannity host hannity moment ago president...,interview karl rove,sean hannity host hannity moments ago presiden...


Word embedding for right wing media

In [4]:
# creating two different datasets for media split by ideology
ideology = df.groupby("ideology")
left = ideology.get_group('left')

In [6]:
left_list = left['text_lemma'].tolist()

In [7]:
sentences_left = []
for article in left_list:
    if isinstance(article, str) and article.strip():
        article_sentences = sent_tokenize(article)
        sentences_left.extend(article_sentences)


In [8]:
import string

# Define a translation table that maps punctuation characters to None
translator = str.maketrans('', '', string.punctuation)

# Iterate through each sentence in sentence_list and remove punctuation
sentences_left = [sentence.translate(translator) for sentence in sentences_left]


In [ ]:
sentences_left

In [ ]:
# left wing
trans_l = str.maketrans('', '', string.punctuation) # translation scheme for removing punctuation
uniquesentences_l = set()
for article in tqdm(list_left):
    for sentence in sent_tokenize(article):
        # remove HTML tags in there
        sentence = re.sub(r"<.*?>"," ",sentence)
        sentence = sentence.translate(trans_l) 
        if sentence not in uniquesentences_l:
            uniquesentences_l.add(sentence.lower())

print(f"We now have {len(uniquesentences_l)} unique sentences.")

In [11]:
import random
import gensim
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
import string
import re

In [9]:
tokenizedsentences_l = [sentence.split() for sentence in sentences_left] #for now, i changed this from a generator to a list (memory seems not really problematic with this small dataset anyway)

In [12]:
# Create an empty list to store the models
models = []

# Define the number of models to create
num_models = 100

# Define the size of the resampled sentences
sample_size = len(tokenizedsentences_l)

# Loop through the number of models to create
for i in range(num_models):
    
    # Use random.choices to resample sentences with replacement
    sample = random.choices(tokenizedsentences_l, k=sample_size)
    
    # Train a Word2Vec model on the resampled sentences
    model_l = gensim.models.Word2Vec(vector_size=300)
    model_l.build_vocab(sample)
    model_l.train(sample, total_examples=model_l.corpus_count, epochs=1)
    
    # Add the trained model to the list of models
    models.append(model_l)

2023-08-30 17:24:09,535 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.025>', 'datetime': '2023-08-30T17:24:09.535283', 'gensim': '4.2.0', 'python': '3.10.8 (tags/v3.10.8:aaaf517, Oct 11 2022, 16:50:30) [MSC v.1933 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}
2023-08-30 17:24:09,535 : INFO : collecting all words and their counts
2023-08-30 17:24:09,535 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


2023-08-30 17:24:09,584 : INFO : PROGRESS: at sentence #10000, processed 126386 words, keeping 10756 word types
2023-08-30 17:24:09,640 : INFO : PROGRESS: at sentence #20000, processed 251146 words, keeping 13548 word types
2023-08-30 17:24:09,704 : INFO : PROGRESS: at sentence #30000, processed 376448 words, keeping 15138 word types
2023-08-30 17:24:09,768 : INFO : PROGRESS: at sentence #40000, processed 502497 words, keeping 16215 word types
2023-08-30 17:24:09,841 : INFO : PROGRESS: at sentence #50000, processed 628970 words, keeping 16968 word types
2023-08-30 17:24:09,930 : INFO : PROGRESS: at sentence #60000, processed 755747 words, keeping 17519 word types
2023-08-30 17:24:09,994 : INFO : PROGRESS: at sentence #70000, processed 881342 words, keeping 17910 word types
2023-08-30 17:24:10,058 : INFO : PROGRESS: at sentence #80000, processed 1010555 words, keeping 18260 word types
2023-08-30 17:24:10,114 : INFO : PROGRESS: at sentence #90000, processed 1139204 words, keeping 18528 w

In [13]:
for i, model in enumerate(models):
    print(f"Top 20 words associated with 'identification' in model {i}:")
    print(model.wv.most_similar('identification', topn=20))
    print("\n")

Top 20 words associated with 'identification' in model 0:
[('require', 0.9775571823120117), ('strict', 0.9747265577316284), ('requirement', 0.9661723971366882), ('stringent', 0.9654963612556458), ('stiffen', 0.9622372388839722), ('effect', 0.9528583884239197), ('governmentissue', 0.9507415294647217), ('photoid', 0.9488078951835632), ('voterid', 0.9465764760971069), ('photo', 0.944073498249054), ('strictest', 0.9420817494392395), ('proofofcitizenship', 0.9410495162010193), ('roll', 0.938807487487793), ('form', 0.9374459385871887), ('acceptable', 0.9374160170555115), ('registration', 0.9372599124908447), ('card', 0.9372377395629883), ('inactive', 0.9346410632133484), ('obtain', 0.9338435530662537), ('driverus', 0.9333617687225342)]


Top 20 words associated with 'identification' in model 1:
[('require', 0.9758013486862183), ('strict', 0.9730836153030396), ('requirement', 0.960803210735321), ('onerous', 0.951940655708313), ('compliant', 0.949637234210968), ('strictest', 0.9460358023643494

In [29]:
from collections import defaultdict

# Create a dictionary to count the frequency of each word
word_count = defaultdict(int)

# Iterate over each model
for model in models:
    # Get the top 20 words associated with "identification"
    top_words = model.wv.most_similar("identification", topn=50)
    
    # Iterate over each top word and add its count to the dictionary
    for word, score in top_words:
        word_count[word] += 1

# Sort the dictionary by descending order of frequency
sorted_word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

# Print the top 10 most common words
for word, count in sorted_word_count[:20]:
    print(word, count)

require 100
strict 100
requirement 100
effect 100
governmentissue 100
photo 100
form 100
registration 100
card 100
purge 100
driver 100
license 99
roll 98
acceptable 98
ids 98
law 95
voterid 93
automatic 93
discriminatory 92
impose 88


In [31]:
top_words2 = ""
for word, count in sorted_word_count[:20]:
    top_words2 += f"{word}, "

print(f"Top 10 most common words: {top_words2[:-2]}")

Top 10 most common words: require, strict, requirement, effect, governmentissue, photo, form, registration, card, purge, driver, license, roll, acceptable, ids, law, voterid, automatic, discriminatory, impose


In [17]:
import numpy as np
similarity_identification_fraud = [model.wv.similarity("identification", "fraud") for model in models]
print(f"mean of simarity between voting and fraud:  {np.mean(similarity_identification_fraud)}")
print(f"std of simarity between voting and fraud:  {np.std(similarity_identification_fraud)}")

mean of simarity between voting and fraud:  0.8551342487335205
std of simarity between voting and fraud:  0.015836862847208977


In [19]:
targets = ['identification']
attributes = ['require', 'strict', 'requirement', 'effect', 'governmentissue', 'photo', 'form', 'registration', 'card', 'purge','driver','license','acceptable','law','roll']
my_results = []
for t in targets:
    for a in attributes:
        combination = f"{a}" 
        my_results.append({'word' : combination, 
                      'scores' : [model.wv.similarity(t, a) for model in models] ,
                      'mean' : np.mean([model.wv.similarity(t, a) for model in models]),
                       'sd' :  np.std([model.wv.similarity(t, a) for model in models])})

In [32]:
# get the results:
for x in my_results:
    print(f"{x['word']} : M = {x['mean']}, SD = {x['sd']}")

require : M = 0.9727168083190918, SD = 0.004084866959601641
strict : M = 0.9746851921081543, SD = 0.004351136740297079
requirement : M = 0.9641630053520203, SD = 0.004833405837416649
effect : M = 0.9412847757339478, SD = 0.008788395673036575
governmentissue : M = 0.9649965167045593, SD = 0.007268586661666632
photo : M = 0.9412448406219482, SD = 0.008383198641240597
form : M = 0.9362974762916565, SD = 0.008083297871053219
registration : M = 0.9324874877929688, SD = 0.008207549341022968
card : M = 0.9319398999214172, SD = 0.009333152323961258
purge : M = 0.9352343678474426, SD = 0.009511173702776432
driver : M = 0.9331060647964478, SD = 0.01029482763260603
license : M = 0.9267376065254211, SD = 0.008265204727649689
acceptable : M = 0.9356984496116638, SD = 0.014001578092575073
law : M = 0.9266941547393799, SD = 0.009300928562879562
roll : M = 0.9279934763908386, SD = 0.009803910739719868


In [33]:
import pandas as pd
# create dataframe
df = pd.DataFrame(my_results)

# select columns and print
df = df[['word', 'mean', 'sd']]
df['mean'] = df['mean'].round(3)
df['sd'] = df['sd'].round(2)
df


,word,mean,sd
0,require,0.973,0.00
1,strict,0.975,0.00
2,requirement,0.964,0.00
3,effect,0.941,0.01
4,governmentissue,0.965,0.01
5,photo,0.941,0.01
6,form,0.936,0.01
7,registration,0.932,0.01
8,card,0.932,0.01
9,purge,0.935,0.01


In [ ]:
df.to_csv('C:/Users/2146806A/OneDrive - University of Glasgow/University of Glasgow/Amsterdam Visit/March_analysis/we_l.csv', index=False)